In [1]:
import h2o
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from time import time

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 day 14 hours 12 mins
H2O cluster version:,3.13.0.3978
H2O cluster version age:,4 years and 9 days !!!
H2O cluster name:,H2O_started_from_R_Sandipan.Dey_kpl973
H2O cluster total nodes:,1
H2O cluster free memory:,2.543 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [3]:
url = "http://coursera.h2o.ai/cacao.882.csv"
cacao = h2o.import_file(url,destination_frame = "Cacao")
cacao

Parse progress: |█████████████████████████████████████████████████████████| 100%


Maker,Origin,REF,Review Date,Cocoa Percent,Maker Location,Rating,Bean Type,Bean Origin
A. Morin,Agua Grande,1876,2016,0.63,France,3.75,Â,Sao Tome
A. Morin,Kpime,1676,2015,0.7,France,2.75,Â,Togo
A. Morin,Atsane,1676,2015,0.7,France,3,Â,Togo
A. Morin,Akata,1680,2015,0.7,France,3.5,Â,Togo
A. Morin,Quilla,1704,2015,0.7,France,3.5,Â,Peru
A. Morin,Carenero,1315,2014,0.7,France,2.75,Criollo,Venezuela
A. Morin,Cuba,1315,2014,0.7,France,3.5,Â,Cuba
A. Morin,Sur del Lago,1315,2014,0.7,France,3.5,Criollo,Venezuela
A. Morin,Puerto Cabello,1319,2014,0.7,France,3.75,Criollo,Venezuela
A. Morin,Pablino,1319,2014,0.7,France,4,Â,Peru


In [7]:
cacao.describe()

Rows:1795
Cols:9




,Maker,Origin,REF,Review Date,Cocoa Percent,Maker Location,Rating,Bean Type,Bean Origin
type,enum,enum,int,int,real,enum,real,enum,enum
mins,,,5.0,2006.0,0.42,,1.0,,
mean,,,1035.9047353760463,2012.325348189414,0.7169832869080773,,3.185933147632313,,
maxs,,,1952.0,2017.0,1.0,,5.0,,
sigma,,,552.8863650101442,2.92721027766861,0.06323118338429189,,0.4780623935863306,,
zeros,,,0,0,0,,0,,
missing,0,0,0,0,0,0,0,1,1
0,A. Morin,Agua Grande,1876.0,2016.0,0.63,France,3.75,Â,Sao Tome
1,A. Morin,Kpime,1676.0,2015.0,0.7000000000000001,France,2.75,Â,Togo
2,A. Morin,Atsane,1676.0,2015.0,0.7000000000000001,France,3.0,Â,Togo


In [8]:
train,test,valid = cacao.split_frame(ratios=[0.7,0.15],destination_frames = ['train_cacao','test_cacao','valid_cacao'],seed=420)

In [9]:
x_cols = ['REF','Cocoa Percent','Maker Location','Bean Type','Bean Origin']
y_col = 'Rating'

In [11]:
from h2o.estimators import H2ODeepLearningEstimator
start = time()
dl = H2ODeepLearningEstimator(seed=1234)
dl.train(training_frame = train,x = x_cols,y=y_col,validation_frame=test)
print("Time taken = ",time()-start," secs")
perf_train = dl.model_performance(train)
perf_test = dl.model_performance(test)
perf_valid = dl.model_performance(valid)

deeplearning Model Build progress: |██████████████████████████████████████| 100%
Time taken =  3.8047678470611572  secs


In [12]:
dl.scoring_history()

,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_deviance,training_mae,validation_rmse,validation_deviance,validation_mae
0,,2021-08-13 02:29:56,0.000 sec,None,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2021-08-13 02:29:56,0.554 sec,3222 obs/sec,1.0,1,1276.0,0.478909,0.229354,0.385200,0.469010,0.219970,0.386216
2,,2021-08-13 02:29:59,3.498 sec,3976 obs/sec,10.0,10,12760.0,0.449333,0.201900,0.355606,0.500892,0.250892,0.392566
3,,2021-08-13 02:29:59,3.643 sec,3963 obs/sec,10.0,10,12760.0,0.478909,0.229354,0.385200,0.469010,0.219970,0.386216


In [13]:
perf_train


ModelMetricsRegression: deeplearning
** Reported on test data. **

MSE: 0.22935427506444606
RMSE: 0.47890946437134235
MAE: 0.3852000535532169
RMSLE: 0.11885485400053404
Mean Residual Deviance: 0.22935427506444606


In [14]:
perf_test


ModelMetricsRegression: deeplearning
** Reported on test data. **

MSE: 0.21997009814241808
RMSE: 0.4690096994118758
MAE: 0.386216223750279
RMSLE: 0.11673122743650532
Mean Residual Deviance: 0.21997009814241808


In [15]:
perf_valid


ModelMetricsRegression: deeplearning
** Reported on test data. **

MSE: 0.20328184322399315
RMSE: 0.4508678777912584
MAE: 0.36578662847753873
RMSLE: 0.11224947937968141
Mean Residual Deviance: 0.20328184322399315


In [16]:
#Tuning
start2 = time()
dl_tuned = H2ODeepLearningEstimator(epochs = 50,hidden=[64,64,64,64],distribution='gaussian',
                                    adaptive_rate=True,
                                    score_training_samples=50,
                                    activation = "MaxoutWithDropout",seed=1234)
dl_tuned.train(training_frame = train,x = x_cols,y=y_col,validation_frame=test)
print("Time Taken = ",time()-start2," secs")
perf_train_v2 = dl_tuned.model_performance(train)
perf_test_v2 = dl_tuned.model_performance(test)
perf_valid_v2 = dl_tuned.model_performance(valid)

deeplearning Model Build progress: |██████████████████████████████████████| 100%
Time Taken =  8.82381820678711  secs


In [17]:
dl_tuned.scoring_history()

,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_deviance,training_mae,validation_rmse,validation_deviance,validation_mae
0,,2021-08-13 02:30:23,0.000 sec,None,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2021-08-13 02:30:24,1.331 sec,5216 obs/sec,5.0,1,6380.0,0.446914,0.199732,0.366826,0.471130,0.221964,0.382119
2,,2021-08-13 02:30:30,6.944 sec,7523 obs/sec,40.0,8,51040.0,0.421889,0.177990,0.353723,0.464446,0.215710,0.375374
3,,2021-08-13 02:30:32,8.701 sec,7503 obs/sec,50.0,10,63800.0,0.407085,0.165718,0.340480,0.446158,0.199057,0.362531


In [18]:
perf_train_v2


ModelMetricsRegression: deeplearning
** Reported on test data. **

MSE: 0.19411645502353408
RMSE: 0.44058648983319276
MAE: 0.35260849849550135
RMSLE: 0.1122210054550642
Mean Residual Deviance: 0.19411645502353408


In [19]:
perf_test_v2


ModelMetricsRegression: deeplearning
** Reported on test data. **

MSE: 0.19905678085221287
RMSE: 0.446157798152417
MAE: 0.36253069218817846
RMSLE: 0.11319528519499306
Mean Residual Deviance: 0.19905678085221287


In [20]:
perf_valid_v2


ModelMetricsRegression: deeplearning
** Reported on test data. **

MSE: 0.18053333867480648
RMSE: 0.424892149462433
MAE: 0.3387604151202593
RMSLE: 0.10832749813639425
Mean Residual Deviance: 0.18053333867480648


In [21]:
#Save models
# path = r".\."
# 1. Baseline model
baseline_model = h2o.save_model(dl, force=True)
#2. Tuned model
tuned_model = h2o.save_model(dl_tuned, force=True)